## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pymorphy2

In [3]:
import pandas as pd
import numpy as np
import pymorphy2
import string

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
train = pd.read_csv("/content/drive/MyDrive/news_train.txt", sep="\t", header=None, names=['topic', 'headline', 'info'])
test = pd.read_csv("/content/drive/MyDrive/news_test.txt", sep="\t", header=None, names=['topic', 'headline', 'info'])

In [5]:
#encode labels
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['topic'])
test['label'] = label_encoder.transform(test['topic'])

In [6]:
def process_text(text):
    # delete punctuation
    punctuation = string.punctuation + '«' + '»' + '—'
    text = "".join(ch if ch not in punctuation else ' ' for ch in text)
    # delete digits
    text = "".join([ch if not ch.isdigit() else ' ' for ch in text])
    #lemmatization
    words = text.split()
    tokens = []
    for word in words:
        word = morphy.parse(word)[0].normal_form;
        #delete stop words 
        if word not in stopwords.words("russian"):
            tokens.append(word)
    return " ".join(tokens)

In [ ]:
morphy = pymorphy2.MorphAnalyzer()
for i in range(len(train)):
    train["info"][i] = process_text(train["info"][i])

for i in range(len(test)):
    test["info"][i] = process_text(test["info"][i])

In [8]:
#Word2Vec
words = [text.split() for text in train["info"]]
model = Word2Vec(sentences=words, min_count=0)
model.wv.most_similar('игра')  

[('игровой', 0.6991193890571594),
 ('серия', 0.693912148475647),
 ('ведьмак', 0.6892495155334473),
 ('китамур', 0.6836866140365601),
 ('консоль', 0.6814697980880737),
 ('дополнение', 0.6809597015380859),
 ('dlc', 0.6758855581283569),
 ('pc', 0.6745489835739136),
 ('геймер', 0.6738331913948059),
 ('dota', 0.6719385385513306)]

In [9]:
#vectoriation
vectorizer = TfidfVectorizer(min_df = 10, max_df = 1., max_features = 10000)
train_vectors = vectorizer.fit_transform(train["info"]).toarray()
test_vectors = vectorizer.transform(test["info"]).toarray()
train_labels = np.array(train['label'])
test_labels = np.array(test['label'])

In [10]:
#SVM
model = SVC(kernel='linear', C=1, gamma = 0.01)
model.fit(train_vectors, train_labels)
print('accuracy: ', accuracy_score(model.predict(test_vectors), test_labels))

accuracy:  0.882


In [11]:
#Multinomial NB
model = MultinomialNB(alpha=0.5)
model.fit(train_vectors, train_labels)
print('accuracy: ', accuracy_score(model.predict(test_vectors), test_labels))

accuracy:  0.8456666666666667
